In [1]:
import pandas as pd
import json
from shapely.geometry import shape
from networkx.readwrite import json_graph
from util import graph_from_geojson

# Czechia population data cleanup

In [2]:
df_cz = pd.read_excel("..\data\cz_population.xlsx", header=3)


In [4]:
df_cz = df_cz.dropna()
df_cz = df_cz[pd.to_numeric(df_cz['celkem'], errors='coerce').notnull()]
df_cz['celkem'] = pd.to_numeric(df_cz['celkem'])
df_cz.rename(columns={'Unnamed: 2': 'name_obec'}, inplace=True)
total_population = df_cz['celkem'].sum()
print("Sum of population in 'celkem' column:", total_population)

Sum of population in 'celkem' column: 10900555


In [5]:
def get_city(station_name, cities):
    possible_cities = [c for c in cities if station_name.startswith(c)]
    if possible_cities:
        return max(possible_cities, key=len)
    return None

In [6]:
city_population = dict(zip(df_cz['name_obec'], df_cz['celkem']))
cities = list(city_population.keys())

In [ ]:
G = graph_from_geojson("../graphs/cze-railroad-network.json")
node_to_city = {}
for node in G.nodes:
    city = get_city(node, cities)
    if city:
        node_to_city[node] = city
    else:
        print(f"Warning: No city match for {node}")


c:\Users\pazera\anaconda3\envs\starter\Lib\site-packages\networkx\readwrite\json_graph\node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


In [ ]:
from collections import Counter
# Count stations per city
city_station_count = Counter(node_to_city.values())
# Assign weights to nodes
node_weights = {}
total_population_used = 0
for node, city in node_to_city.items():
    P_C = city_population[city]  # City population
    N_C = city_station_count[city]  # Number of stations in city
    w_node = P_C / N_C  # Weight for this station
    total_population_used += w_node
    node_weights[node] = w_node
    
print(f"Total population used: {total_population_used}")

# Add weights to graph as node attributes
for node in G.nodes:
    if node in node_weights:
        G.nodes[node]['pop'] = node_weights[node]
    else:
        total_population_used += 1000
        G.nodes[node]['pop'] = 1000 # For unmatched stations

for node in G.nodes:
    # add population probability to each node
    G.nodes[node]['prob'] = G.nodes[node]['pop']/total_population_used
    

Total population used: 7179048.000000003


# Calculation of the ratio of skyline distance against shortest path on the graph for Czechia

In [ ]:
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import pandas as pd
from tqdm import tqdm
import time

nodes = list(G.nodes)
probabilities = [G.nodes[node]['prob'] for node in nodes]
num_iterations = 1_000_000

ratios = []
starting_node = []
finish_node = []
valid_pairs = 0  

start_time = time.time()
for _ in tqdm(range(num_iterations), desc="Processing iterations"):
    # Select two distinct nodes
    u, v = np.random.choice(nodes, size=2, replace=False, p=probabilities)
    
    # Calculate straight-line distance
    point_u = G.nodes[u]['geometry']
    point_v = G.nodes[v]['geometry']
    coords_u = point_u.coords[0]  
    coords_v = point_v.coords[0]
    latlon_u = (coords_u[1], coords_u[0])  
    latlon_v = (coords_v[1], coords_v[0])
    straight_line_distance = geodesic(latlon_u, latlon_v).meters
    
    # Calculate shortest path distance
    try:
        shortest_path_distance = nx.shortest_path_length(G, source=u, target=v, weight='distance')
        # Compute ratio (shortest path distance / straight-line distance)
        if straight_line_distance > 0:  # Avoid division by zero
            ratio = shortest_path_distance / straight_line_distance
            ratios.append(ratio)
            starting_node.append(u)
            finish_node.append(v)
            valid_pairs += 1
    except nx.NetworkXNoPath:
        # Skip pairs with no path (or assign ratio = inf if desired)
        continue

results_df = pd.DataFrame({'ratio': ratios, 'u': starting_node, 'v': finish_node})

output_path = 'generated_data/skyline_vs_shortest/Czech_ratios.csv'
results_df.to_csv(output_path, index=False)

print(f"Total iterations: {num_iterations}")
print(f"Valid pairs (with path): {valid_pairs}")
print(f"Invalid pairs (no path): {num_iterations - valid_pairs}")
if ratios:
    print(f"Mean ratio: {np.mean(ratios):.4f}")
    print(f"Median ratio: {np.median(ratios):.4f}")
    print(f"Min ratio: {np.min(ratios):.4f}")
    print(f"Max ratio: {np.max(ratios):.4f}")
else:
    print("No valid paths found.")
print(f"Results saved to: {output_path}")
print(f"Time taken: {time.time() - start_time:.2f} seconds")

Processing iterations:   1%|▏         | 14868/1000000 [00:18<20:52, 786.41it/s]

In [ ]:
import folium
from shapely.geometry import Point
import networkx as nx

country = 'Slovak'
code = 'svk' 
G = graph_from_geojson(f"../graphs/{code}-railroad-network.json")
results_df = pd.read_csv(f'generated_data/skyline_vs_shortest/{country}_ratios.csv')
selected_pair = results_df.iloc[1523]
print(selected_pair)
u = selected_pair['u']
v = selected_pair['v']
shortest_path = nx.shortest_path(G, source=u, target=v, weight='distance')
path_coords = [(G.nodes[node]['geometry'].y, G.nodes[node]['geometry'].x) for node in shortest_path]
straight_line_coords = [
    (G.nodes[u]['geometry'].y, G.nodes[u]['geometry'].x),
    (G.nodes[v]['geometry'].y, G.nodes[v]['geometry'].x)
]

m = folium.Map()
for node1, node2 in G.edges():
    if node1 in G.nodes and node2 in G.nodes and 'geometry' in G.nodes[node1] and 'geometry' in G.nodes[node2]:
        geom1 = G.nodes[node1]['geometry']
        geom2 = G.nodes[node2]['geometry']
        if isinstance(geom1, Point) and isinstance(geom2, Point):
            lat1, lon1 = geom1.y, geom1.x
            lat2, lon2 = geom2.y, geom2.x
            folium.PolyLine([[lat1, lon1], [lat2, lon2]], color='lightgray', weight=1).add_to(m)

folium.PolyLine(straight_line_coords, color='green', weight=2.5, dash_array='5').add_to(m)
folium.PolyLine(path_coords, color='red', weight=2.5).add_to(m)
folium.Marker(location=straight_line_coords[0], popup=f'Start: {u}', icon=folium.Icon(color='green')).add_to(m)
folium.Marker(location=straight_line_coords[1], popup=f'End: {v}', icon=folium.Icon(color='red')).add_to(m)

all_coords = path_coords + straight_line_coords
lats = [coord[0] for coord in all_coords]
lons = [coord[1] for coord in all_coords]
min_lat, max_lat = min(lats), max(lats)
min_lon, max_lon = min(lons), max(lons)
m.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])

m.save(f'../vizualizations/{country}_shortest_path_example.html')

c:\Users\pazera\anaconda3\envs\starter\Lib\site-packages\networkx\readwrite\json_graph\node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


ratio                3.885443
u        Kokava nad Rimavicou
v            Spišská Nová Ves
Name: 1523, dtype: object


# Slovakia population cleanup

In [10]:
df_sk = pd.read_csv("..\data\sk_population.csv")
display(df_sk)
sum(df_sk["population"])

,name,link,region,district,region_historical,elevation,coordinate_x,coordinate_y,area,population,first_mentioned
0,Ábelová,/wiki/%C3%81belov%C3%A1,Banskobystrický kraj,Lučenec,Novohrad,445.0,48.410833,19.432222,52.17,196,1275
1,Abovce,/wiki/Abovce,Banskobystrický kraj,Rimavská Sobota,Gemer,159.0,48.318889,20.336944,8.20,626,1339
2,Abrahám,/wiki/Abrah%C3%A1m_(okres_Galanta),Trnavský kraj,Galanta,undefined,125.0,48.250000,17.616667,15.78,1055,1231
3,Abrahámovce,/wiki/Abrah%C3%A1movce_(okres_Bardejov),Prešovský kraj,Bardejov,Šariš,267.0,49.161667,21.342222,5.90,341,1427
4,Abrahámovce,/wiki/Abrah%C3%A1movce_(okres_Ke%C5%BEmarok),Prešovský kraj,Kežmarok,undefined,739.0,49.044669,20.437262,6.65,269,1286
...,...,...,...,...,...,...,...,...,...,...,...
2885,Žitavce,/wiki/%C5%BDitavce,Nitriansky kraj,Nitra,undefined,141.0,48.201690,18.299638,8.29,396,1232
2886,Žitná-Radiša,/wiki/%C5%BDitn%C3%A1-Radi%C5%A1a,Trenčiansky kraj,Bánovce nad Bebravou,Horná Nitra,275.0,48.766500,18.344400,17.77,443,1295
2887,Žlkovce,/wiki/%C5%BDlkovce,Trnavský kraj,Hlohovec,Dolné Považie,145.0,48.466667,17.716667,7.94,660,1229
2888,Župčany,/wiki/%C5%BDup%C4%8Dany,Prešovský kraj,Prešov,Šariš,315.0,49.016667,21.166667,8.49,1690,1248


5457936

In [11]:
city_population = dict(zip(df_sk['name'], df_sk['population']))
cities = list(city_population.keys())


In [ ]:
G = graph_from_geojson("../graphs/svk-railroad-network.json")
node_to_city = {}
for node in G.nodes:
    city = get_city(node, cities)
    if city:
        node_to_city[node] = city
    else:
        print(f"Warning: No city match for {node}")


c:\Users\pazera\anaconda3\envs\starter\Lib\site-packages\networkx\readwrite\json_graph\node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


In [ ]:
from collections import Counter
# Count stations per city
city_station_count = Counter(node_to_city.values())
# Assign weights to nodes
node_weights = {}
total_population_used = 0
for node, city in node_to_city.items():
    P_C = city_population[city]  # City population
    N_C = city_station_count[city]  # Number of stations in city
    w_node = P_C / N_C  # Weight for this station
    total_population_used += w_node
    node_weights[node] = w_node
    
print(f"Total population used: {total_population_used}")

# Add weights to graph as node attributes
for node in G.nodes:
    if node in node_weights:
        G.nodes[node]['pop'] = node_weights[node]
    else:
        total_population_used += 1000
        G.nodes[node]['pop'] = 1000 # For unmatched stations

for node in G.nodes:
    G.nodes[node]['prob'] = G.nodes[node]['pop']/total_population_used
    

Total population used: 3096147.9999999995


# Calculation of the ratio of skyline distance against shortest path on the graph for Slovakia


In [ ]:
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import pandas as pd
from tqdm import tqdm
import time

nodes = list(G.nodes)
probabilities = [G.nodes[node]['prob'] for node in nodes]
num_iterations = 1_000_000

ratios = []
starting_node = []
finish_node = []
valid_pairs = 0  

start_time = time.time()
for _ in tqdm(range(num_iterations), desc="Processing iterations"):
    u, v = np.random.choice(nodes, size=2, replace=False, p=probabilities)
    
    # Calculate straight-line distance
    point_u = G.nodes[u]['geometry']
    point_v = G.nodes[v]['geometry']
    coords_u = point_u.coords[0]  # (longitude, latitude)
    coords_v = point_v.coords[0]
    latlon_u = (coords_u[1], coords_u[0])  # (latitude, longitude)
    latlon_v = (coords_v[1], coords_v[0])
    straight_line_distance = geodesic(latlon_u, latlon_v).meters
    
    # Calculate shortest path distance
    try:
        shortest_path_distance = nx.shortest_path_length(G, source=u, target=v, weight='distance')
        # Compute ratio (shortest path distance / straight-line distance)
        if straight_line_distance > 0:  # Avoid division by zero
            ratio = shortest_path_distance / straight_line_distance
            starting_node.append(u)
            finish_node.append(v)
            ratios.append(ratio)
            valid_pairs += 1
    except nx.NetworkXNoPath:
        # Skip pairs with no path (or assign ratio = inf if desired)
        continue

results_df = pd.DataFrame({'ratio': ratios, 'u': starting_node, 'v': finish_node})
output_path = 'generated_data/skyline_vs_shortest/Slovak_ratios.csv'
results_df.to_csv(output_path, index=False)

print(f"Total iterations: {num_iterations}")
print(f"Valid pairs (with path): {valid_pairs}")
print(f"Invalid pairs (no path): {num_iterations - valid_pairs}")
if ratios:
    print(f"Mean ratio: {np.mean(ratios):.4f}")
    print(f"Median ratio: {np.median(ratios):.4f}")
    print(f"Min ratio: {np.min(ratios):.4f}")
    print(f"Max ratio: {np.max(ratios):.4f}")
else:
    print("No valid paths found.")
print(f"Results saved to: {output_path}")
print(f"Time taken: {time.time() - start_time:.2f} seconds")

Processing iterations: 100%|██████████| 1000000/1000000 [16:19<00:00, 1021.07it/s]


Total iterations: 1000000
Valid pairs (with path): 971398
Invalid pairs (no path): 28602
Mean ratio: 1.7941
Median ratio: 1.3172
Min ratio: 1.0000
Max ratio: 375.0650
Results saved to: ../test_skyline_vs_shorthest_path\Slovak_ratios.csv
Time taken: 985.47 seconds


Data from https://github.com/adammertel/municipalities-slovakia/blob/master/out/municipalities-slovakia.csv for Slovakia
and for Czechia from https://csu.gov.cz/produkty/pocet-obyvatel-v-obcich-9vln2prayv